In [2]:
# libraries for numerical and data processing
import pandas as pd
import numpy as np


# ML-specific libraries
import xgboost as xgb
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


# libraries for visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


# helpful util libraries
import random
import json
import os
import joblib
from datetime import datetime, timedelta
from functools import partial


pd.set_option("display.max_columns", 500)


TEMP_DATA_DIR = "temp_data"
TRANSFORMED_DATA_DIR = "transformed_data"
TRANSFORMED_DATA_DIR_v2 = "transformed_data_v2"
GAMELOG_DIR = "gamelog_data"
GAME_ROTATION_DIR = "game_rotation_data"
TEAM_STAT_DIR = "team_stat_data"
PLAYER_STAT_DIR = "player_stat_data"
TRAINING_DIR = "training_data"
TRAINING_DIR_v2 = "training_data_v2"


BUCKET = "mert-kurttutan-nba-project-v1"
BUCKET_TRANSFORMED = "mert-kurttutan-nba-project-transformed-data-v1"

cols_to_drop = ["team1_TEAM_NAME", "team1_TEAM_ID", "team2_TEAM_NAME", "team2_TEAM_ID", 
                "team1_L", "team2_W", "team2_L", "team1_PLUS_MINUS", "team1_PTS", 
                "team2_PLUS_MINUS", "team2_PTS", "team2_IS_HOME"]


cols_to_drop_v2 = ["team1_TEAM_ID", "team2_TEAM_ID", 
                "team1_L", "team2_W", "team2_L", "team1_PLUS_MINUS", "team1_PTS", 
                "team2_PLUS_MINUS", "team2_PTS", "team2_IS_HOME"]

training_files_sorted = sorted([gamelog_file for gamelog_file in os.listdir(f"../../data/{TRANSFORMED_DATA_DIR_v2}/{TRAINING_DIR}")])

training_files_v2_sorted = sorted([gamelog_file for gamelog_file in os.listdir(f"../../data/{TRANSFORMED_DATA_DIR_v2}/{TRAINING_DIR_v2}")])

In [3]:
df_train_arr = []
for idx, train_file in enumerate(training_files_sorted):
    if not ".csv" in train_file:
        continue


    df_gamelog = pd.read_csv(f"../../data/{TRANSFORMED_DATA_DIR_v2}/{TRAINING_DIR}/{train_file}")
    df_gamelog = df_gamelog.drop(columns=cols_to_drop).sort_values(by=["DATE_WITHIN_SEASON"])

    if idx%2 == 0:
      df_gamelog["SEASON"] = idx+1

    else:
      df_gamelog["SEASON"] = idx-1
    
    # Append data of current season into total data
    df_train_arr.append(df_gamelog)


# Get rid of stat of the very early time
# We could not fill this since there is no previos timeline to use
df_train_arr[1] = df_train_arr[1].loc[~df_train_arr[1]["team2_lag08_AST"].isna(), :]

In [4]:
df_train_arr_v2 = []
for idx, train_file in enumerate(training_files_v2_sorted):
    if not ".csv" in train_file:
        continue
    df_gamelog = pd.read_csv(f"../../data/{TRANSFORMED_DATA_DIR_v2}/{TRAINING_DIR_v2}/{train_file}")
    df_gamelog = df_gamelog.drop(columns=cols_to_drop_v2).sort_values(by=["DATE_WITHIN_SEASON"])



    if idx%2 == 0:
      df_gamelog["SEASON"] = idx+1

    else:
      df_gamelog["SEASON"] = idx-1
    
    # Append data of current season into total data
    df_train_arr_v2.append(df_gamelog)


# Get rid of stat of the very early time
# We could not fill this since there is no previos timeline to use
df_train_arr_v2[1] = df_train_arr_v2[1].loc[~df_train_arr_v2[1]["team2_lag08_AST"].isna(), :]

In [5]:
df_train_arr[-1]

,DATE_WITHIN_SEASON,GAME_DATE,IS_REGULAR,team1_IS_HOME,team1_L_cum,team1_W,team1_W_cum,team1_lag08_AST,team1_lag08_BLK,team1_lag08_DREB,team1_lag08_FG_PCT,team1_lag08_FT_PCT,team1_lag08_GP,team1_lag08_OREB,team1_lag08_PLUS_MINUS,team1_lag08_PLUS_MINUS_RANK,team1_lag08_PTS,team1_lag08_PTS_RANK,team1_lag08_REB,team1_lag08_STL,team1_lag08_TOV,team1_lag08_W_PCT,team1_lag08_W_PCT_RANK,team1_lag16_AST,team1_lag16_BLK,team1_lag16_DREB,team1_lag16_FG_PCT,team1_lag16_FT_PCT,team1_lag16_GP,team1_lag16_OREB,team1_lag16_PLUS_MINUS,team1_lag16_PLUS_MINUS_RANK,team1_lag16_PTS,team1_lag16_PTS_RANK,team1_lag16_REB,team1_lag16_STL,team1_lag16_TOV,team1_lag16_W_PCT,team1_lag16_W_PCT_RANK,team1_lag180_AST,team1_lag180_BLK,team1_lag180_DREB,team1_lag180_FG_PCT,team1_lag180_FT_PCT,team1_lag180_GP,team1_lag180_OREB,team1_lag180_PLUS_MINUS,team1_lag180_PLUS_MINUS_RANK,team1_lag180_PTS,team1_lag180_PTS_RANK,team1_lag180_REB,team1_lag180_STL,team1_lag180_TOV,team1_lag180_W_PCT,team1_lag180_W_PCT_RANK,team1_lag32_AST,team1_lag32_BLK,team1_lag32_DREB,team1_lag32_FG_PCT,team1_lag32_FT_PCT,team1_lag32_GP,team1_lag32_OREB,team1_lag32_PLUS_MINUS,team1_lag32_PLUS_MINUS_RANK,team1_lag32_PTS,team1_lag32_PTS_RANK,team1_lag32_REB,team1_lag32_STL,team1_lag32_TOV,team1_lag32_W_PCT,team1_lag32_W_PCT_RANK,team1_lag64_AST,team1_lag64_BLK,team1_lag64_DREB,team1_lag64_FG_PCT,team1_lag64_FT_PCT,team1_lag64_GP,team1_lag64_OREB,team1_lag64_PLUS_MINUS,team1_lag64_PLUS_MINUS_RANK,team1_lag64_PTS,team1_lag64_PTS_RANK,team1_lag64_REB,team1_lag64_STL,team1_lag64_TOV,team1_lag64_W_PCT,team1_lag64_W_PCT_RANK,team2_L_cum,team2_W_cum,team2_lag08_AST,team2_lag08_BLK,team2_lag08_DREB,team2_lag08_FG_PCT,team2_lag08_FT_PCT,team2_lag08_GP,team2_lag08_OREB,team2_lag08_PLUS_MINUS,team2_lag08_PLUS_MINUS_RANK,team2_lag08_PTS,team2_lag08_PTS_RANK,team2_lag08_REB,team2_lag08_STL,team2_lag08_TOV,team2_lag08_W_PCT,team2_lag08_W_PCT_RANK,team2_lag16_AST,team2_lag16_BLK,team2_lag16_DREB,team2_lag16_FG_PCT,team2_lag16_FT_PCT,team2_lag16_GP,team2_lag16_OREB,team2_lag16_PLUS_MINUS,team2_lag16_PLUS_MINUS_RANK,team2_lag16_PTS,team2_lag16_PTS_RANK,team2_lag16_REB,team2_lag16_STL,team2_lag16_TOV,team2_lag16_W_PCT,team2_lag16_W_PCT_RANK,team2_lag180_AST,team2_lag180_BLK,team2_lag180_DREB,team2_lag180_FG_PCT,team2_lag180_FT_PCT,team2_lag180_GP,team2_lag180_OREB,team2_lag180_PLUS_MINUS,team2_lag180_PLUS_MINUS_RANK,team2_lag180_PTS,team2_lag180_PTS_RANK,team2_lag180_REB,team2_lag180_STL,team2_lag180_TOV,team2_lag180_W_PCT,team2_lag180_W_PCT_RANK,team2_lag32_AST,team2_lag32_BLK,team2_lag32_DREB,team2_lag32_FG_PCT,team2_lag32_FT_PCT,team2_lag32_GP,team2_lag32_OREB,team2_lag32_PLUS_MINUS,team2_lag32_PLUS_MINUS_RANK,team2_lag32_PTS,team2_lag32_PTS_RANK,team2_lag32_REB,team2_lag32_STL,team2_lag32_TOV,team2_lag32_W_PCT,team2_lag32_W_PCT_RANK,team2_lag64_AST,team2_lag64_BLK,team2_lag64_DREB,team2_lag64_FG_PCT,team2_lag64_FT_PCT,team2_lag64_GP,team2_lag64_OREB,team2_lag64_PLUS_MINUS,team2_lag64_PLUS_MINUS_RANK,team2_lag64_PTS,team2_lag64_PTS_RANK,team2_lag64_REB,team2_lag64_STL,team2_lag64_TOV,team2_lag64_W_PCT,team2_lag64_W_PCT_RANK,SEASON
0,0,2021-10-19,1,1,0.0,1,0.0,0.29,0.10,0.65,0.422,0.750,1.0,0.21,-0.13,20.0,2.33,19.0,0.85,0.27,0.25,0.00,16.0,0.29,0.10,0.65,0.422,0.750,1.0,0.21,-0.13,20.0,2.33,19.0,0.85,0.27,0.25,0.000,16.0,0.53,0.10,0.79,0.488,0.765,58.0,0.21,0.10,6.0,2.49,1.0,1.00,0.16,0.28,0.638,6.0,0.29,0.10,0.65,0.422,0.750,1.0,0.21,-0.13,20.0,2.33,19.0,0.85,0.27,0.25,0.000,16.0,0.29,0.10,0.65,0.422,0.750,1.0,0.21,-0.13,20.0,2.33,19.0,0.85,0.27,0.25,0.000,16.0,0.0,0.0,0.73,0.19,0.69,0.566,0.765,1.0,0.08,0.29,5.0,2.56,6.0,0.77,0.19,0.25,1.00,1.0,0.73,0.19,0.69,0.566,0.765,1.0,0.08,0.29,5.0,2.56,6.0,0.77,0.19,0.25,1.000,1.0,0.56,0.11,0.72,0.494,0.804,57.0,0.19,0.09,7.0,2.45,2.0,0.91,0.14,0.27,0.684,5.0,0.73,0.19,0.69,0.566,0.765,1.0,0.08,0.29,5.0,2.56,6.0,0.77,0.19,0.25,1.000,1.0,0.73,0.19,0.69,0.566,0.765,1.0,0.08,0.29,5.0,2.56,6.0,0.77,0.19,0.25,1.000,1.0,31
1,0,2021-10-19,1,0,0.0,1,0.0,0.54,0.08,0.71,

In [ ]:
df_train_arr_v2[0]

,DATE_WITHIN_SEASON,IS_REGULAR,team1_IS_HOME,team1_L_cum,team1_PStat_lag08_AST,team1_PStat_lag08_AST_RANK,team1_PStat_lag08_BLK,team1_PStat_lag08_DREB,team1_PStat_lag08_DREB_RANK,team1_PStat_lag08_FG3M,team1_PStat_lag08_FG3M_RANK,team1_PStat_lag08_FG3_PCT,team1_PStat_lag08_FG3_PCT_RANK,team1_PStat_lag08_FGM_RANK,team1_PStat_lag08_FG_PCT,team1_PStat_lag08_FG_PCT_RANK,team1_PStat_lag08_FTM,team1_PStat_lag08_FT_PCT,team1_PStat_lag08_FT_PCT_RANK,team1_PStat_lag08_GP,team1_PStat_lag08_NBA_FANTASY_PTS,team1_PStat_lag08_NBA_FANTASY_PTS_RANK,team1_PStat_lag08_OREB,team1_PStat_lag08_OREB_RANK,team1_PStat_lag08_PLUS_MINUS,team1_PStat_lag08_PLUS_MINUS_RANK,team1_PStat_lag08_PTS,team1_PStat_lag08_PTS_RANK,team1_PStat_lag08_STL,team1_PStat_lag08_STL_RANK,team1_PStat_lag08_TOV,team1_PStat_lag08_TOV_RANK,team1_PStat_lag08_W_PCT,team1_PStat_lag08_W_PCT_RANK,team1_PStat_lag16_AST,team1_PStat_lag16_AST_RANK,team1_PStat_lag16_BLK,team1_PStat_lag16_DREB,team1_PStat_lag16_DREB_RANK,team1_PStat_lag16_FG3M,team1_PStat_lag16_FG3M_RANK,team1_PStat_lag16_FG3_PCT,team1_PStat_lag16_FG3_PCT_RANK,team1_PStat_lag16_FGM_RANK,team1_PStat_lag16_FG_PCT,team1_PStat_lag16_FG_PCT_RANK,team1_PStat_lag16_FTM,team1_PStat_lag16_FT_PCT,team1_PStat_lag16_FT_PCT_RANK,team1_PStat_lag16_GP,team1_PStat_lag16_NBA_FANTASY_PTS,team1_PStat_lag16_NBA_FANTASY_PTS_RANK,team1_PStat_lag16_OREB,team1_PStat_lag16_OREB_RANK,team1_PStat_lag16_PLUS_MINUS,team1_PStat_lag16_PLUS_MINUS_RANK,team1_PStat_lag16_PTS,team1_PStat_lag16_PTS_RANK,team1_PStat_lag16_STL,team1_PStat_lag16_STL_RANK,team1_PStat_lag16_TOV,team1_PStat_lag16_TOV_RANK,team1_PStat_lag16_W_PCT,team1_PStat_lag16_W_PCT_RANK,team1_PStat_lag180_AST,team1_PStat_lag180_AST_RANK,team1_PStat_lag180_BLK,team1_PStat_lag180_DREB,team1_PStat_lag180_DREB_RANK,team1_PStat_lag180_FG3M,team1_PStat_lag180_FG3M_RANK,team1_PStat_lag180_FG3_PCT,team1_PStat_lag180_FG3_PCT_RANK,team1_PStat_lag180_FGM_RANK,team1_PStat_lag180_FG_PCT,team1_PStat_lag180_FG_PCT_RANK,team1_PStat_lag180_FTM,team1_PStat_lag180_FT_PCT,team1_PStat_lag180_FT_PCT_RANK,team1_PStat_lag180_GP,team1_PStat_lag180_NBA_FANTASY_PTS,team1_PStat_lag180_NBA_FANTASY_PTS_RANK,team1_PStat_lag180_OREB,team1_PStat_lag180_OREB_RANK,team1_PStat_lag180_PLUS_MINUS,team1_PStat_lag180_PLUS_MINUS_RANK,team1_PStat_lag180_PTS,team1_PStat_lag180_PTS_RANK,team1_PStat_lag180_STL,team1_PStat_lag180_STL_RANK,team1_PStat_lag180_TOV,team1_PStat_lag180_TOV_RANK,team1_PStat_lag180_W_PCT,team1_PStat_lag180_W_PCT_RANK,team1_PStat_lag32_AST,team1_PStat_lag32_AST_RANK,team1_PStat_lag32_BLK,team1_PStat_lag32_DREB,team1_PStat_lag32_DREB_RANK,team1_PStat_lag32_FG3M,team1_PStat_lag32_FG3M_RANK,team1_PStat_lag32_FG3_PCT,team1_PStat_lag32_FG3_PCT_RANK,team1_PStat_lag32_FGM_RANK,team1_PStat_lag32_FG_PCT,team1_PStat_lag32_FG_PCT_RANK,team1_PStat_lag32_FTM,team1_PStat_lag32_FT_PCT,team1_PStat_lag32_FT_PCT_RANK,team1_PStat_lag32_GP,team1_PStat_lag32_NBA_FANTASY_PTS,team1_PStat_lag32_NBA_FANTASY_PTS_RANK,team1_PStat_lag32_OREB,team1_PStat_lag32_OREB_RANK,team1_PStat_lag32_PLUS_MINUS,team1_PStat_lag32_PLUS_MINUS_RANK,team1_PStat_lag32_PTS,team1_PStat_lag32_PTS_RANK,team1_PStat_lag32_STL,team1_PStat_lag32_STL_RANK,team1_PStat_lag32_TOV,team1_PStat_lag32_TOV_RANK,team1_PStat_lag32_W_PCT,team1_PStat_lag32_W_PCT_RANK,team1_PStat_lag64_AST,team1_PStat_lag64_AST_RANK,team1_PStat_lag64_BLK,team1_PStat_lag64_DREB,team1_PStat_lag64_DREB_RANK,team1_PStat_lag64_FG3M,team1_PStat_lag64_FG3M_RANK,team1_PStat_lag64_FG3_PCT,team1_PStat_lag64_FG3_PCT_RANK,team1_PStat_lag64_FGM_RANK,team1_PStat_lag64_FG_PCT,team1_PStat_lag64_FG_PCT_RANK,team1_PStat_lag64_FTM,team1_PStat_lag64_FT_PCT,team1_PStat_lag64_FT_PCT_RANK,team1_PStat_lag64_GP,team1_PStat_lag64_NBA_FANTASY_PTS,team1_PStat_lag64_NBA_FANTASY_PTS_RANK,team1_PStat_lag64_OREB,team1_PStat_lag64_OREB_RANK,team1_PStat_lag64_PLUS_MINUS,team1_PStat_lag64_PLUS_MINUS_RANK,team1_PStat_lag64_PTS,team1_PStat_lag64_PTS_RANK,team1_PStat_lag64_STL,team1_PStat_lag64_STL_RANK,team1_PStat_lag64_TOV,team1_PS

In [ ]:
team1_labels = []
team2_labels = []
for i in list(df_train_arr[0].columns):
  if "team1" in i:
    team1_labels.append(i)
  elif "team2" in i:
    team2_labels.append(i)


for i in team1_labels:
  if not f"team2{i[5:]}" in team2_labels:
    print(i)

team1_IS_HOME
team1_W


In [ ]:
team1_labels = []
team2_labels = []
for i in list(df_train_arr_v2[0].columns):
  if "team1" in i:
    team1_labels.append(i)
  elif "team2" in i:
    team2_labels.append(i)


for i in team1_labels:
  if not f"team2{i[5:]}" in team2_labels:
    print(i)

team1_IS_HOME
team1_W


In [ ]:
train = pd.concat(df_train_arr[:-2])
val = pd.concat(df_train_arr[-2:])

In [ ]:
data_v2 = pd.concat(df_train_arr_v2).sort_values(by=["SEASON", "DATE_WITHIN_SEASON"])

In [ ]:
pandemic_regular_idx = data_v2["DATE_WITHIN_SEASON"] > 200

In [ ]:
max_within_season = data_v2.loc[pandemic_regular_idx, "DATE_WITHIN_SEASON"].max()

In [ ]:
def shift_to(x, end_value: int = 180, max_value: int = max_within_season):
  return x - max_value + end_value

In [ ]:
data_v2.loc[pandemic_regular_idx, "DATE_WITHIN_SEASON"] = shift_to(data_v2.loc[pandemic_regular_idx, "DATE_WITHIN_SEASON"])

In [ ]:
fig, axs = plt.subplots(len(train.columns)-1, figsize=(10, 600))
n = 0
for col in train.columns:
  if col == "GAME_DATE":
    continue
  
  axs[n].set_title(col)
  train[col].hist(ax=axs[n], bins=100)
  n += 1



In [ ]:
fig, axs = plt.subplots(len(data_v2.columns)//2 +1, 2, figsize=(12, 800))

for idx, col in enumerate(data_v2.columns):
  
  idx_1 = idx // 2 
  idx_2 = idx % 2
  axs[idx_1, idx_2].set_title(col)
  axs[idx_1, idx_2].hist(data_v2_normalized[:,idx], bins=100)

  data_v2[col].hist(ax=axs[idx_1, idx_2], bins=100)



In [ ]:
# Set figure size
plt.figure(figsize=(20,20))

# Calculate correlation matrix
cor = data_v2.corr() 

# Plot the correlation matrix
#sns.heatmap(cor, annot=True, cmap=plt.cm.PuBu)
#plt.show()

<Figure size 1440x1440 with 0 Axes>

In [ ]:
# Get the absolute value of the correlation
cor_target = abs(cor["team1_W"])

# Select highly correlated features (thresold = 0.2)
relevant_features = cor_target[cor_target>0.05]

# Collect the names of the features
names = [index for index, value in relevant_features.iteritems()]

# Drop the target variable from the results
#names.remove('team1_W')

# Display the results
print(names, len(names))

names += ["SEASON", "DATE_WITHIN_SEASON"]

['team1_IS_HOME', 'team1_L_cum', 'team1_PStat_lag08_FG_PCT_RANK', 'team1_PStat_lag08_PLUS_MINUS', 'team1_PStat_lag08_PLUS_MINUS_RANK', 'team1_PStat_lag08_W_PCT', 'team1_PStat_lag08_W_PCT_RANK', 'team1_PStat_lag16_FG3M_RANK', 'team1_PStat_lag16_FG_PCT', 'team1_PStat_lag16_FG_PCT_RANK', 'team1_PStat_lag16_PLUS_MINUS', 'team1_PStat_lag16_PLUS_MINUS_RANK', 'team1_PStat_lag16_W_PCT', 'team1_PStat_lag16_W_PCT_RANK', 'team1_PStat_lag180_AST', 'team1_PStat_lag180_FG3M_RANK', 'team1_PStat_lag180_FG3_PCT_RANK', 'team1_PStat_lag180_FG_PCT', 'team1_PStat_lag180_FG_PCT_RANK', 'team1_PStat_lag180_NBA_FANTASY_PTS', 'team1_PStat_lag180_NBA_FANTASY_PTS_RANK', 'team1_PStat_lag180_PLUS_MINUS', 'team1_PStat_lag180_PLUS_MINUS_RANK', 'team1_PStat_lag180_PTS', 'team1_PStat_lag180_PTS_RANK', 'team1_PStat_lag180_W_PCT', 'team1_PStat_lag180_W_PCT_RANK', 'team1_PStat_lag32_FG3M_RANK', 'team1_PStat_lag32_FG3_PCT_RANK', 'team1_PStat_lag32_FG_PCT', 'team1_PStat_lag32_FG_PCT_RANK', 'team1_PStat_lag32_NBA_FANTASY_PTS

In [ ]:
data_v2 = data_v2[names]

In [ ]:
def objective_hyperparam(trial):

    # Dictionary to map from parameter name to values
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 64, 1024),
        'eta': trial.suggest_float("eta", 0.001, 0.9, log=True),
        'max_depth': trial.suggest_int("max_depth", 1, 15, step=1),
        'min_child_weight': trial.suggest_float("min_child_weight", 0.3, 1.2, step=0.1),
        #'eval_metric': ["binary:logistic"],
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.3, 0.9, step=0.1), 
        'subsample':trial.suggest_float("subsample", 0.3, 0.9, step=0.1), 
        #'early_stopping_rounds': 15,
        'seed': trial.suggest_int("seed", 42, 43),
        'verbose' : False,
    }
    
    # XGboost model
    model = xgb.XGBClassifier(
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        min_child_weight=params['min_child_weight'], 
        colsample_bytree=params['colsample_bytree'], 
        subsample=params['subsample'], 
        eta=params['eta'],
        tree_method='gpu_hist',
        seed=params['seed'],
        n_jobs=4)
    
    split_num = 1
    
    total_score = 0

    # Calculate score of model by taking weighted average over 5 splits
    for i in range(split_num):
        

        regular_idx = (data_v2["SEASON"]%2 == 0) #& (data_v2["SEASON"] > 10)


        val_idx = (data_v2["DATE_WITHIN_SEASON"] > 155) #& (data_v2["SEASON"]%4 == 0) #& (data_v2["SEASON"] == 28)
        train_idx = ~val_idx

        X_train = data_v2.loc[regular_idx&train_idx,:]
        y_train = X_train.pop("team1_W")


        X_val = data_v2.loc[regular_idx&val_idx,:]
        y_val = X_val.pop("team1_W")

        # Apply min-max normalization
        #scaler = MinMaxScaler().fit(X_train)
        #scaler = StandardScaler().fit(X_train)
        #X_train = scaler.transform(X_train)
        #X_val = scaler.transform(X_val)


        model.fit(
            X_train, 
            y_train, 
            #eval_metric=params["eval_metric"], 
            eval_set=[(X_train, y_train), (X_val,y_val)], 
            verbose=params['verbose'], 
            early_stopping_rounds = 8)


        # Predictions
        yhat_val = model.predict(X_val)

        # Accuracy
        accuracy = (yhat_val == y_val).sum() / len(y_val)

        
        total_score += accuracy
    
    # Take the averegge
    total_score = total_score / split_num
    
    return total_score

In [ ]:
# Study object to optimize objective_hyperparam function
study_01 = optuna.create_study(direction="maximize")
study_01.optimize(objective_hyperparam, n_trials=10)

version = "v1"
joblib.dump(study_01, f"./study_xgboost_hyparam_{version}.hyp")

[I 2022-06-09 10:53:42,850] A new study created in memory with name: no-name-156f5fbb-5860-4640-8e48-a95bda692a78
[I 2022-06-09 10:53:44,025] Trial 0 finished with value: 0.6559422193040052 and parameters: {'n_estimators': 554, 'eta': 0.1943565351394589, 'max_depth': 11, 'min_child_weight': 0.9000000000000001, 'colsample_bytree': 0.9, 'subsample': 0.4, 'seed': 42}. Best is trial 0 with value: 0.6559422193040052.
[I 2022-06-09 10:53:45,620] Trial 1 finished with value: 0.6585686145764937 and parameters: {'n_estimators': 95, 'eta': 0.006285919719410498, 'max_depth': 13, 'min_child_weight': 0.7, 'colsample_bytree': 0.8, 'subsample': 0.6000000000000001, 'seed': 42}. Best is trial 1 with value: 0.6585686145764937.
[I 2022-06-09 10:53:45,937] Trial 2 finished with value: 0.6664478003939593 and parameters: {'n_estimators': 712, 'eta': 0.002022583558858939, 'max_depth': 5, 'min_child_weight': 1.2, 'colsample_bytree': 0.4, 'subsample': 0.8, 'seed': 43}. Best is trial 2 with value: 0.66644780039

['./study_01.pkl']

In [ ]:
study_hyper = joblib.load('./study_01.pkl')

In [ ]:
study_hyper.best_params

{'colsample_bytree': 0.7,
 'eta': 0.05820858995258255,
 'max_depth': 4,
 'min_child_weight': 0.9000000000000001,
 'n_estimators': 411,
 'seed': 43,
 'subsample': 0.8}

In [ ]:
# Dictionary to map from parameter name to values
params = study_hyper.best_params

params["verbose"] = False

# XGboost model
model = xgb.XGBClassifier(
    max_depth=params['max_depth'],
    n_estimators=params['n_estimators'],
    min_child_weight=params['min_child_weight'], 
    colsample_bytree=params['colsample_bytree'], 
    subsample=params['subsample'], 
    eta=params['eta'],
    tree_method='gpu_hist',
    seed=params['seed'],
    n_jobs=4)

split_num = 1

total_score = 0

season_arr = [22, 24, 26, 28]

# Calculate score of model by taking weighted average over 5 splits
for i in range(split_num):
    

    regular_idx = data_v2["SEASON"].isin(season_arr)


    val_idx = (data_v2["DATE_WITHIN_SEASON"] > 155) & (data_v2["SEASON"] == season_arr[-1])
    train_idx = ~val_idx

    X_train = data_v2.loc[regular_idx&train_idx,:]
    y_train = X_train.pop("team1_W")


    X_val = data_v2.loc[regular_idx&val_idx,:]
    y_val = X_val.pop("team1_W")

    # Apply min-max normalization
    #scaler = MinMaxScaler().fit(X_train)
    #scaler = StandardScaler().fit(X_train)
    #X_train = scaler.transform(X_train)
    #X_val = scaler.transform(X_val)


    model.fit(
        X_train, 
        y_train, 
        #eval_metric=params["eval_metric"], 
        eval_set=[(X_train, y_train), (X_val,y_val)], 
        verbose=params['verbose'],)
        #early_stopping_rounds = 8)


    # Predictions
    yhat_val = model.predict(X_val)

    # Accuracy
    accuracy = (yhat_val == y_val).sum() / len(y_val)

    
    total_score += accuracy

# Take the averegge
total_score = total_score / split_num

In [ ]:
# XGboost model
model = xgb.XGBClassifier(
    max_depth=params['max_depth'],
    n_estimators=params['n_estimators'],
    min_child_weight=params['min_child_weight'], 
    colsample_bytree=params['colsample_bytree'], 
    subsample=params['subsample'], 
    eta=params['eta'],
    tree_method='gpu_hist',
    seed=params['seed'],
    n_jobs=4,
    early_stop=8)


season_arr = [12, 14, 16, 18]
val_end = int(data_v2.loc[data_v2["SEASON"]==season_arr[-1], "DATE_WITHIN_SEASON"].max())
val_start = int(val_end*0.90)
print(val_end)

val_day_arr = list(range(val_start+1, val_start+2))

total_score = 0
total_preds = 0

# Calculate score of model by taking weighted average over 5 splits
for val_day in val_day_arr:
    #print(f"Current val_day: {val_day}")


    regular_idx = (data_v2["SEASON"].isin(season_arr))


    val_idx = (data_v2["DATE_WITHIN_SEASON"] >= val_day) & (data_v2["SEASON"] == season_arr[-1])
    train_idx = (data_v2["DATE_WITHIN_SEASON"] < val_day) | (data_v2["SEASON"] != season_arr[-1])

    X_train = data_v2.loc[regular_idx&train_idx,:]
    y_train = X_train.pop("team1_W")


    X_val = data_v2.loc[regular_idx&val_idx,:]
    y_val = X_val.pop("team1_W")
    
    if X_val.shape[0] < 1:
      print("No game is played at this day, and skipping to the next day...")
      continue


    model.fit(
        X_train, 
        y_train, 
        #eval_metric=params["eval_metric"], 
        eval_set=[(X_train, y_train), (X_val,y_val)], 
        verbose=params['verbose'],)


    # Predictions
    yhat_val = model.predict(X_val)

    # Number of correct pred
    current_score = (yhat_val == y_val).sum()
    total_score += current_score
    
    # Number of preds
    current_preds = len(y_val)
    total_preds += current_preds


    
# Take the average
total_acc = total_score / total_preds

169


In [ ]:
model.save_model("model_01.xgb")

In [ ]:
reg_xgb = xgb.Booster()
reg_xgb.load_model("model_01.xgb")

In [ ]:
print(total_acc)

0.6296296296296297


In [ ]:
data_v2.loc[regular_idx,:]

,team1_IS_HOME,team1_L_cum,team1_PStat_lag08_FG_PCT_RANK,team1_PStat_lag08_PLUS_MINUS,team1_PStat_lag08_PLUS_MINUS_RANK,team1_PStat_lag08_W_PCT,team1_PStat_lag08_W_PCT_RANK,team1_PStat_lag16_FG3M_RANK,team1_PStat_lag16_FG_PCT,team1_PStat_lag16_FG_PCT_RANK,team1_PStat_lag16_PLUS_MINUS,team1_PStat_lag16_PLUS_MINUS_RANK,team1_PStat_lag16_W_PCT,team1_PStat_lag16_W_PCT_RANK,team1_PStat_lag180_AST,team1_PStat_lag180_FG3M_RANK,team1_PStat_lag180_FG3_PCT_RANK,team1_PStat_lag180_FG_PCT,team1_PStat_lag180_FG_PCT_RANK,team1_PStat_lag180_NBA_FANTASY_PTS,team1_PStat_lag180_NBA_FANTASY_PTS_RANK,team1_PStat_lag180_PLUS_MINUS,team1_PStat_lag180_PLUS_MINUS_RANK,team1_PStat_lag180_PTS,team1_PStat_lag180_PTS_RANK,team1_PStat_lag180_W_PCT,team1_PStat_lag180_W_PCT_RANK,team1_PStat_lag32_FG3M_RANK,team1_PStat_lag32_FG3_PCT_RANK,team1_PStat_lag32_FG_PCT,team1_PStat_lag32_FG_PCT_RANK,team1_PStat_lag32_NBA_FANTASY_PTS,team1_PStat_lag32_NBA_FANTASY_PTS_RANK,team1_PStat_lag32_PLUS_MINUS,team1_PStat_lag32_PLUS_MINUS_RANK,team1_PStat_lag32_W_PCT,team1_PStat_lag32_W_PCT_RANK,team1_PStat_lag64_FG3M_RANK,team1_PStat_lag64_FG3_PCT_RANK,team1_PStat_lag64_FG_PCT,team1_PStat_lag64_FG_PCT_RANK,team1_PStat_lag64_NBA_FANTASY_PTS,team1_PStat_lag64_NBA_FANTASY_PTS_RANK,team1_PStat_lag64_PLUS_MINUS,team1_PStat_lag64_PLUS_MINUS_RANK,team1_PStat_lag64_PTS,team1_PStat_lag64_W_PCT,team1_PStat_lag64_W_PCT_RANK,team1_W,team1_W_cum,team1_lag08_AST,team1_lag08_FG_PCT,team1_lag08_PLUS_MINUS,team1_lag08_PLUS_MINUS_RANK,team1_lag08_PTS,team1_lag08_PTS_RANK,team1_lag08_W_PCT,team1_lag08_W_PCT_RANK,team1_lag16_AST,team1_lag16_BLK,team1_lag16_FG_PCT,team1_lag16_PLUS_MINUS,team1_lag16_PLUS_MINUS_RANK,team1_lag16_PTS,team1_lag16_PTS_RANK,team1_lag16_W_PCT,team1_lag16_W_PCT_RANK,team1_lag180_AST,team1_lag180_BLK,team1_lag180_DREB,team1_lag180_FG_PCT,team1_lag180_PLUS_MINUS,team1_lag180_PLUS_MINUS_RANK,team1_lag180_PTS,team1_lag180_PTS_RANK,team1_lag180_TOV,team1_lag180_W_PCT,team1_lag180_W_PCT_RANK,team1_lag32_AST,team1_lag32_BLK,team1_lag32_FG_PCT,team1_lag32_PLUS_MINUS,team1_lag32_PLUS_MINUS_RANK,team1_lag32_PTS,team1_lag32_PTS_RANK,team1_lag32_TOV,team1_lag32_W_PCT,team1_lag32_W_PCT_RANK,team1_lag64_AST,team1_lag64_BLK,team1_lag64_FG_PCT,team1_lag64_PLUS_MINUS,team1_lag64_PLUS_MINUS_RANK,team1_lag64_PTS,team1_lag64_PTS_RANK,team1_lag64_STL,team1_lag64_TOV,team1_lag64_W_PCT,team1_lag64_W_PCT_RANK,team2_L_cum,team2_PStat_lag08_AST,team2_PStat_lag08_FG3M,team2_PStat_lag08_FG_PCT,team2_PStat_lag08_FG_PCT_RANK,team2_PStat_lag08_PLUS_MINUS,team2_PStat_lag08_PLUS_MINUS_RANK,team2_PStat_lag08_PTS,team2_PStat_lag08_W_PCT,team2_PStat_lag08_W_PCT_RANK,team2_PStat_lag16_AST,team2_PStat_lag16_FG3M,team2_PStat_lag16_FG_PCT,team2_PStat_lag16_FG_PCT_RANK,team2_PStat_lag16_NBA_FANTASY_PTS,team2_PStat_lag16_NBA_FANTASY_PTS_RANK,team2_PStat_lag16_PLUS_MINUS,team2_PStat_lag16_PLUS_MINUS_RANK,team2_PStat_lag16_PTS,team2_PStat_lag16_W_PCT,team2_PStat_lag16_W_PCT_RANK,team2_PStat_lag180_AST,team2_PStat_lag180_FG3M,team2_PStat_lag180_FG3_PCT,team2_PStat_lag180_FG_PCT,team2_PStat_lag180_FG_PCT_RANK,team2_PStat_lag180_NBA_FANTASY_PTS,team2_PStat_lag180_NBA_FANTASY_PTS_RANK,team2_PStat_lag180_PLUS_MINUS,team2_PStat_lag180_PLUS_MINUS_RANK,team2_PStat_lag180_PTS,team2_PStat_lag180_W_PCT,team2_PStat_lag180_W_PCT_RANK,team2_PStat_lag32_AST,team2_PStat_lag32_FG3M,team2_PStat_lag32_FG3_PCT,team2_PStat_lag32_FG_PCT,team2_PStat_lag32_FG_PCT_RANK,team2_PStat_lag32_NBA_FANTASY_PTS,team2_PStat_lag32_NBA_FANTASY_PTS_RANK,team2_PStat_lag32_PLUS_MINUS,team2_PStat_lag32_PLUS_MINUS_RANK,team2_PStat_lag32_PTS,team2_PStat_lag32_W_PCT,team2_PStat_lag32_W_PCT_RANK,team2_PStat_lag64_AST,team2_PStat_lag64_FG3M,team2_PStat_lag64_FG3_PCT,team2_PStat_lag64_FG_PCT,team2_PStat_lag64_FG_PCT_RANK,team2_PStat_lag64_NBA_FANTASY_PTS,team2_PStat_lag64_NBA_FANTASY_PTS_RANK,team2_PStat_lag64_PLUS_MINUS,team2_PStat_lag64_PLUS_MINUS_RANK,team2_PStat_lag64_PTS,team2_PStat_lag64_W_PCT,team2_PStat_lag64_W_PCT_RANK,team2_W_cum,team2_lag08_AST,team2